In [1]:
import warnings; warnings.simplefilter('ignore')
from IPython.display import HTML

css_adjustment = """
<style>
/* Cacher les prompts de commande */
div.prompt {display: none !important;}

/* Ajuster la largeur max des cellules de code et de markdown */
.jp-RenderedMarkdown, .jp-InputArea {
    max-width: 1500px !important; /* Assurez-vous que cela correspond à la largeur du Markdown si nécessaire */
}

/* Modifier la taille de la police dans les cellules de code */
.jp-Notebook .jp-InputArea .input_area {
    font-size: 40px !important; /* Ajustez selon vos besoins */
}

/* Si vous souhaitez ajuster la taille de la police à l'intérieur des blocs de code eux-mêmes */
.jp-Notebook .jp-InputArea .input_area pre, 
.jp-Notebook .jp-InputArea .input_area code {
    font-size: 40px !important; /* Ajustez selon vos besoins */
}
</style>
"""

HTML(css_adjustment)

from IPython.display import display, HTML

# Programmer avec des LLM

## Objectif de ces Travaux Pratiques

- Courte introduction à "comment programmer avec des LLM"
- Uniquement de la pratique (pas le temps pour la théorie)
- Evaluation : 
  - Projet de votre choix de programmation par des LLM
  - Note = fonction(quantite_de_travail)

## Introduction

### LangChain
- Framework Python d'intégration de modèles LLM
  - Model agnostic 
  - Chaînes et Agents
  - Automatisation des prompts
  - Gestion des vectorstores, embeddings...

- Cas d’usage : chatbots (RAG), programmes embarquant des LLM, LangGraphs, multi-agents...

### Mistral AI

1. Créez un compte sur La Plateforme de Mistral AI. https://mistral.ai/

   ("Try the API" -> "S'inscrire")

   (Abonnement : Gratuit / expérimental)
   
3. Générez une clé API personnelle.

   (API -> Clés API -> Créer une nouvelle clé -> ...)

### Environnement virtuel

Il y aura potentiellement des bibliothèques à installer, ce qui peut se faire dans un environnement virtuel python.

  $ python -m venv mon\_env
  
  $ source mon\_env/bin/activate
  
  (mon\_env) $ pip install nom\_bibliotheque

(Et pour désactiver : deactivate)

### Références
- LangChain
  - https://python.langchain.com/docs/introduction/
  - https://www.youtube.com/@LangChain
- Mistral AI
  - https://docs.mistral.ai/
- OpenAI
  - https://platform.openai.com

## Introduction à LangChain / Mistral

### Premières invocations

In [2]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import HumanMessage

# Initialiser le modèle
llm = ChatMistralAI(model="mistral-medium-2508", 
                    temperature=0)
                    #api_key="....")

# Créer le message utilisateur
message = HumanMessage(content="Quelle est la capitale de l'Albanie ?")

# Obtenir la réponse
response = llm.invoke([message])

# Afficher la réponse
print(response.content)

La capitale de l'Albanie est Tirana.


In [3]:
# Version OpenAI
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4.1")
response = llm.invoke("Quelle est la capitale de l'Albanie ?")
print(response.content)

La capitale de l’Albanie est Tirana.


In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

# 1) Initialisation du modèle Mistral
model = ChatMistralAI(model="mistral-large-latest", 
                      temperature=0)

# 2) Construction du prompt + parser
prompt = ChatPromptTemplate.from_template("Fais-moi une blague sur le sujet : {sujet}")
output_parser = StrOutputParser()

# 3) Chaînage (LangChain Expression Language)
chain = prompt | model | output_parser

# 4) Exécution
for sujet in ['pompier', 'police']:
    print(chain.invoke({"sujet": sujet}))
    print('-'*10)


Pourquoi les pompiers ne jouent-ils jamais à cache-cache ?

Parce qu'ils trouvent toujours la cachette en feu !
----------
Pourquoi les policiers n'aiment-ils pas les puzzles ?

Parce qu'ils préfèrent les casse-têtes !
----------


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Vous êtes un rédacteur de documentation technique de classe mondiale."),
    ("user", "{input}")
])

llm = ChatMistralAI(model="mistral-large-latest")
chain = prompt | llm
result = chain.invoke({"input": "Qu'est-ce que le modèle mistral-large-latest ?"})

In [6]:
result.usage_metadata

{'input_tokens': 38, 'output_tokens': 701, 'total_tokens': 739}

In [18]:
38/1000000*2+701*8/100000

0.056156

In [19]:
print(result.content)

Mistral-large-latest est un modèle de langage développé par Mistral AI, une société de pointe dans le domaine de l'intelligence artificielle. Ce modèle est conçu pour comprendre et générer du texte en réponse à une variété d'entrées, ce qui le rend utile pour diverses applications telles que la rédaction de documentation technique, la traduction, la génération de contenu, et bien plus encore.

### Caractéristiques principales de Mistral-large-latest :

1. **Taille et Capacité** :
   - Le modèle est de grande taille, ce qui lui permet de comprendre et de générer du texte de manière très sophistiquée.
   - Il est entraîné sur une vaste quantité de données textuelles, ce qui lui confère une large compréhension du langage naturel.

2. **Polyvalence** :
   - Mistral-large-latest peut être utilisé pour une variété de tâches, y compris la rédaction de documents techniques, la réponse à des questions, la génération de contenu créatif, et même la traduction entre différentes langues.

3. **Préc

## Les sorties structurées

### Cas d'un booléen

In [8]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

class Answer(BaseModel):
    answer: bool

prompt_answer = [
    ("system", "Tu es un assistant chargé de répondre un booléen (True ou False) à la question d'un utilisateur."),
    ('human', "{question}")
]

prompt_answer_template = ChatPromptTemplate.from_messages(prompt_answer)
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)
chain = prompt_answer_template | llm.with_structured_output(schema=Answer)

def repond(question):
    return chain.invoke({"question": question}).answer
    
for question in ["Noël est en hiver", "Il pleut quand il pleut pas"]:
    print(question)
    reponse = repond(question)
    print(f"Réponse : {reponse} (type : {type(reponse)})")
    print()


Noël est en hiver
Réponse : True (type : <class 'bool'>)

Il pleut quand il pleut pas
Réponse : False (type : <class 'bool'>)



### Cas d'une classe

In [9]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

tasks = ["Répondre à une nouvelle question", "Fournir plus d'éléments à la question précédente"]

class NextTask(BaseModel):
    """Utilise toujours cet outil pour structurer ta réponse to the user."""
    action: str = Field(..., 
                        enum=tasks,
                        description="La prochaine action à mener")

prompt_message = [
    ("system", "Tu es un assistant chargé de classifier la demande d'un utilisateur parmi une "
               "liste réduite d'actions à mener en tant que chatbot. Tu dois déterminer la "
               "prochaine action à mener."),
    ('human', "{text}")
]

prompt = ChatPromptTemplate.from_messages(prompt_message)
llm = ChatMistralAI(model='mistral-large-latest', temperature=0)
chain = prompt | llm.with_structured_output(schema=NextTask)

for text in ["Peux-tu m'en dire plus", "Que sont les PPV ?"]:
    print(text)
    print(chain.invoke({"text": text}))
    print()

Peux-tu m'en dire plus
action="Fournir plus d'éléments à la question précédente"

Que sont les PPV ?
action='Répondre à une nouvelle question'



### Cas d'un entier

In [10]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

class TonMessage(BaseModel):
    """Évaluation du ton du message de l'utilisateur."""
    note_ton: int = Field(
        ..., 
        ge=1,
        le=5,
        description="Note attribuée au ton du message : 1 pour neutre, 5 pour très aimable"
    )

prompt_message = [
    ("system", "Tu es un assistant chargé d'évaluer le ton d'un message donné par l'utilisateur. "
               "Attribue une note de 1 à 5 au ton du message, où 1 signifie neutre et 5 signifie très aimable."),
    ('human', "{text}")
]

prompt = ChatPromptTemplate.from_messages(prompt_message)
llm = ChatMistralAI(model='mistral-large-latest', temperature=0)
chain = prompt | llm.with_structured_output(schema=TonMessage)

messages = [
    "Bonjour, pourrais-tu m'aider s'il te plaît ?",
    "J'ai besoin de ça immédiatement.",
    "Merci beaucoup pour ton aide précieuse !"
]

for text in messages:
    print(f"Message : {text}")
    print(chain.invoke({"text": text}))
    print()

Message : Bonjour, pourrais-tu m'aider s'il te plaît ?
note_ton=4

Message : J'ai besoin de ça immédiatement.
note_ton=1

Message : Merci beaucoup pour ton aide précieuse !
note_ton=5



### Pourquoi et comment forcer la sortie du LLM

In [11]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import HumanMessage

llm = ChatMistralAI(model="mistral-medium-latest")
message = HumanMessage(content="Peux-tu me traduire ce qui suit, en anglais ?\n\n Quelle est la capitale de l'Albanie ?")
llm.invoke([message])

AIMessage(content='The translation of "Quelle est la capitale de l\'Albanie ?" in English is:\n\n**"What is the capital of Albania?"**', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 26, 'total_tokens': 56, 'completion_tokens': 30}, 'model_name': 'mistral-medium-latest', 'model': 'mistral-medium-latest', 'finish_reason': 'stop'}, id='run--c4b5c2c7-5a43-4f1d-a93e-0ed2170a7818-0', usage_metadata={'input_tokens': 26, 'output_tokens': 30, 'total_tokens': 56})

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_mistralai.chat_models import ChatMistralAI
from pydantic import BaseModel, Field

class Translation(BaseModel):
    original_text: str = Field(..., description="The original text before translation in another language")
    original_language: str = Field(..., description="The original language before translation")
    translated_text: str = Field(..., description="The final text after translation in another language")
    translated_language: str = Field(..., description="The language into which the translation must be done")
    
def traduit(texte, langue_source="français", langue_cible="anglais"):
    llm = ChatMistralAI(model_name="mistral-medium-latest")
    prompt = ChatPromptTemplate.from_template("""Je souhaite que tu traduises le texte suivant du {langue_source} vers le {langue_cible}. Ta traduction doit être précise, fluide et naturelle, et préserver parfaitement le sens original. 
    Retourne-moi la réponse sous forme d'objet JSON avec les champs :
      - original_text : le texte original
      - original_language : la langue du texte original
      - translated_text : la traduction du texte
      - translated_language : la langue de la traduction

    Voici le texte à traduire :
    ----
    {texte}""")
    output_parser = StrOutputParser()
    extract_translation = RunnableLambda(lambda translation: translation.translated_text)
    chain0 = prompt | llm.with_structured_output(Translation) | extract_translation
    return chain0.invoke({"langue_source": langue_source,
                            "langue_cible": "anglais",
                            "texte": texte      
                            })

print(traduit("Quelle est la capitale de l'Albanie"))

What is the capital of Albania


## Des sorties structurées aux prémices d'un raisonnement

In [13]:
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

class Etape(BaseModel):
    explication: str
    sortie: str

class MathReponse(BaseModel):
    etapes: list[Etape]
    reponse_finale: str

prompt_answer = [
    ("system", "Tu es un professeur de mathématiques très pédagogue."),
    ('human', "{exercice}")
]

prompt_answer_template = ChatPromptTemplate.from_messages(prompt_answer)
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)
chain = prompt_answer_template | llm.with_structured_output(schema=MathReponse)

In [14]:
explications = chain.invoke({"exercice": "Résous  8x + 31 = 2"})
for etape in explications.etapes:
    print(f"- {etape.explication}")
    print(f"  Le résultat est alors : {etape.sortie}")

print(f"Au final, on trouve : {explications.reponse_finale}")

- Soustrayons 31 des deux côtés de l'équation pour isoler le terme en x.
  Le résultat est alors : 8x + 31 - 31 = 2 - 31
8x = -29
- Divisons les deux côtés par 8 pour résoudre x.
  Le résultat est alors : 8x / 8 = -29 / 8
x = -29/8 ou x = -3,625
Au final, on trouve : x = -3,625


## Retrieval-Augmented Generation

<div style="text-align: center;">
    <img src="images/rag.png" alt="RAG">
</div>

### Embeddings et semantique

But : encoder un texte sous la forme d'un vecteur, de sorte que deux textes voisins sémantiquement soient encodés en deux vecteurs proches.

![Texte alternatif](images/vectors-and-semantics.png "Vectors")

### Embeddings : Bag of words

![Texte alternatif](images/Bag-of-words.png "BoW")

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'Demonstration text, first document',
    "Demo text, and here's a second document.",
    'And finally, this is the third document.'
]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

print("Vocabulary :", vectorizer.get_feature_names_out())
print("BoW vector:\n", X.toarray())

Vocabulary : ['and' 'demo' 'demonstration' 'document' 'finally' 'first' 'here' 'is'
 'second' 'text' 'the' 'third' 'this']
BoW vector:
 [[0 0 1 1 0 1 0 0 0 1 0 0 0]
 [1 1 0 1 0 0 1 0 1 1 0 0 0]
 [1 0 0 1 1 0 0 1 0 0 1 1 1]]


### Embeddings par transformers

In [1]:
# pip install sentence-transformers
from sentence_transformers import SentenceTransformer

sentences = ["This is an example sentence.", "Each sentence is converted into a fixed-sized vector."]

# Entraîné sur des données essentiellement anglophones.
# Conçu pour être léger et rapide, tout en gardant une bonne précision pour l’anglais.
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

for sentence, embedding in zip(sentences, embeddings):
    print(f'"{sentence}" -> {embedding[:3]}...')

print(f"Embedding size: {len(embedding)}")

RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from sentence_transformers import SentenceTransformer

#Entraîné avec un objectif de détection de paraphrases sur un corpus multilingue.
#Performances équilibrées pour la similarité sémantique, la recherche d’information et la classification zero-shot en plusieurs langues.

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print(model.encode(["Texte à encoder"]))

### Similarité sémantique

In [24]:
import numpy as np

def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    return dot_product / (norm_A * norm_B)

cosine_similarity(embeddings[0], embeddings[1])

np.float32(0.37034684)

### Embeddings OpenAI

In [28]:
from openai import OpenAI

openai = OpenAI()

def embed(text, model="text-embedding-3-large", dimensions=3072): #3072: dimension maximale
    return openai.embeddings.create(input = [text], model=model, dimensions=dimensions).data[0].embedding

vector1 = embed("What is Mycobacterium kansasii ?")
vector2 = embed("To sum up, we have presented a case of Mycobacterium kansasii monoarthritis of the elbow complicated with unusual clinical and radiological findings. A combination of synovectomy and multidrug antimycobacterial treatment yielded a favorable clinical course without recurrence of arthritis after 10 months of follow-up. This case emphasizes the need to consider this rare infection in the differential diagnosis of intra-articular soft tissue tumor-like lesions of the elbow even in immunocompetent patients.")
cosine_similarity(vector1, vector2)

np.float64(0.5604925298797377)

### RAG : principe de base

<div style="text-align: center;">
    <img src="images/rag2.png" alt="RAG">
</div>

In [27]:
from langchain_mistralai.chat_models import ChatMistralAI

llm = ChatMistralAI(model_name="mistral-large-latest")

query = "What is Mycobacterium kansasii ?"
context = "To sum up, we have presented a case of Mycobacterium kansasii monoarthritis of the elbow complicated with unusual clinical and radiological findings. A combination of synovectomy and multidrug antimycobacterial treatment yielded a favorable clinical course without recurrence of arthritis after 10 months of follow-up. This case emphasizes the need to consider this rare infection in the differential diagnosis of intra-articular soft tissue tumor-like lesions of the elbow even in immunocompetent patients."

text = f"""You are an expert in the Mycobacterium field. 
Answer to the following question by only using the context below.

question: {query}

context : {context}"""

response = llm.invoke(text)
print(response.content)

Mycobacterium kansasii


### Implémentation d'un vectorstore

In [31]:
#pip install langchain-community langchain-openai faiss-cpu

import warnings; warnings.simplefilter('ignore')
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

loader = PyPDFLoader("images/Guyeux_2024.pdf")
pages = loader.load_and_split()

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faiss_index = FAISS.from_documents(pages, embeddings)
docs = faiss_index.similarity_search("Is there a lineage 10 in M.tuberculosis?", k=2)

In [32]:
from textwrap import shorten, fill

for doc in docs:
    print(f"Page {doc.metadata["page"]}: {fill(shorten(doc.page_content, 500), 80)}\n")

Page 3: M. africanum Lineage 10, Central Africa Conclusions Through the extensive mining
of WGS and genotyp- ing databases, we newly identified a thus far rare M.
tuberculosis complex lineage, L10 (proposed), pres- ent in central Africa. The
lineage is characterized by a new region of deletion, IS6110 insertions, and 243
SNPs, including gyrA G7901T, recN C1920096T, and dnaG C2621730T. L10 represents
a sister clade to L6, found mainly in western Africa, and L9, specifically in
eastern Africa, and [...]

Page 0: nity of Lille, Lille, France (P. Supply, C. Gaudin); London School of Hygiene
and Tropical Medicine, London, UK (J.E. Phelan, T.G. Clark, L. Rigouts, B. de
Jong); Université Paris-Saclay, Saint- Aubin, France (C. Sola); Université Paris
Cité, Paris (C. Sola) DOI: https://doi.org/10.3201/eid3003.231466 Analysis of
genome sequencing data from >100,000 genomes of Mycobacterium tuberculosis
complex using TB-Annotator software revealed a previously unknown lineage,
proposed name L10, 

### Version OpenAI

In [ ]:
#pip install langchain-community langchain-openai faiss-cpu
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = PyPDFLoader("images/Guyeux_2024.pdf")
pages = loader.load_and_split()

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("Is there a lineage 10 in M.tuberculosis?", k=2)

### Text splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text = '''Vous pouvez partager un article en cliquant sur les icônes de partage en haut à droite de celui-ci. 
La reproduction totale ou partielle d’un article, sans l’autorisation écrite et préalable du Monde, est strictement interdite. 
Pour plus d’informations, consultez nos conditions générales de vente. 

Comme la finance, la politique est parfois affaire d’opportunités. Aux Etats-Unis, l’opposition démocrate à Donald Trump a en tout cas trouvé un nouvel angle d’attaque après l’annonce par le président américain d’une pause dans sa guerre commerciale : elle le soupçonne d’avoir manipulé les marchés boursiers et d’avoir ainsi favorisé des délits d’initié.
Lire aussi | Article réservé à nos abonnés Droits de douane : les Bourses rechutent, l’inquiétude s’étend aux emprunts d’Etat

Le sénateur Adam Schiff a écrit, jeudi 10 avril, au directeur par intérim du Bureau pour l’éthique gouvernementale (Office of Government Ethics, OGE), une agence fédérale indépendante, et à Susan Wiles, la cheffe de cabinet de la Maison Blanche, pour leur demander d’ouvrir une enquête « urgente » afin de déterminer si « le président Trump, sa famille ou d’autres membres de [son] administration » ont commis la veille des délits d’initié en profitant d’informations confidentielles sur le revirement de sa politique commerciale.

'''

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    keep_separator=False,
    separators=["\n\n", "\n", ". "]
)

texts = text_splitter.create_documents([text])

for k in texts[:7]:
    print(k.page_content)
    print("="*20+'\n')


### Loaders (LangChain)

In [ ]:
from langchain_community.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=YcIbZGTRMjI", 
    language=['fr'],
    add_video_info=False
)

print(loader.load())

### Vectorstores

Nombreux et multiples...
 - FAISS, Chroma : faciles à maîtriser, déployer...
 - Milvus : multi-embeddings, BM25, filtrage par colonne...

<div style="text-align: center;">
    <img src="images/Milvus.png" alt="RAG">
</div>

## Les agents

Un agent est un système autonome alimenté par un modèle de langage (comme GPT-4) qui prend des décisions sur les actions à entreprendre en fonction des données d'entrée et des instructions programmées.

Fonction :
- Prise de décision : L'agent analyse les données d'entrée et utilise des algorithmes et des modèles pour décider quelle action entreprendre.
- Exécution d'actions : L'agent peut effectuer diverses actions comme répondre à une question, rechercher des informations, ou interagir avec d'autres systèmes.

Un "agent", c'est un LLM avec des "outils" :
 - recherche sur internet,
 - calculatrice,
 - interrogation de pdf (RAG),
 - outil fait maison
 - ...

Le mieux est de faire des agents spécialisés, et de les orchestrer ensemble.

### Des outils

#### Wikipedia

In [ ]:
#%pip install --upgrade --quiet  wikipedia
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wikipedia.run("Alan Turing")

#### Tavili (recherche internet)

In [ ]:
# pip install -qU langchain-tavily
# Pour une clé d'API : https://www.tavily.com/
from langchain_community.tools.tavily_search import TavilySearchResults

import os
os.environ["TAVILY_API_KEY"] = "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"

search = TavilySearchResults(max_results=2)
search_results = search.invoke("Quel est le temps à Belfort ?")
print(search_results[0]['content'][:100])

### Agents LangChain

#### Exécuteur d'agent (Agent Executor)

L'exécuteur d'agent est un composant ou un système qui orchestre et exécute les actions déterminées par l'agent.

Fonction :
 - Gestion de l'exécution : Il reçoit les décisions de l'agent, exécute les actions correspondantes et gère la transition entre différentes étapes de l'exécution.
 - Traitement des résultats : Il collecte les résultats des actions exécutées et les transmet à l'agent pour de nouvelles décisions ou à l'utilisateur final.

Exemple : Dans un système de recommandation, l'exécuteur d'agent pourrait orchestrer l'appel à différentes API pour recueillir les informations nécessaires (comme les préférences de l'utilisateur et les données sur les produits) et les combiner pour générer une recommandation.

#### Avec outil Tavily

In [ ]:
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.messages import HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_mistralai.chat_models import ChatMistralAI

search = TavilySearchResults(max_results=2)
tools = [search]

llm = ChatMistralAI(model="mistral-large-latest", temperature=0)

prompt = hub.pull("amalnuaimi/react-mistral")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke(
    {
        'input': "Dois-je prendre un parapluie, sachant que je me rends aujourd'hui et demain à Belfort ?",
        'chat_history': []
    })
print(response['output'])

In [ ]:
#https://smith.langchain.com/hub
prompt.pretty_print()

In [ ]:
# Version OpenAI
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.messages import HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

search = TavilySearchResults(max_results=2)
tools = [search]

llm = ChatOpenAI(model_name="gpt-4.1")

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke(
    {
        'input': "Dois-je prendre un parapluie, sachant que je me rends aujourd'hui et demain à Belfort ?"
    })
print(response['output'])

#### Avec outil arXiv

In [ ]:
# pip install arxiv
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain_mistralai.chat_models import ChatMistralAI

#llm = ChatOpenAI(temperature=0.0)
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)
tools = load_tools(["arxiv"])

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Résume l'article 1605.08386 en français"})

In [ ]:
prompt.pretty_print()

In [ ]:
print(tools[0].name)
print(tools[0].description)

#### Avec Python REPL

In [ ]:
from langchain.agents import create_openai_functions_agent
from langchain_experimental.tools import PythonREPLTool

tools = [PythonREPLTool()]

instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""

base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
agent = create_openai_functions_agent(ChatOpenAI(temperature=0), tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Quel est le millième nombre de Fibonacci ?"})

In [ ]:
base_prompt.pretty_print()

#### Plusieurs outils

In [ ]:
from langchain.agents import load_tools, create_react_agent
from langchain_openai import OpenAI

llm = OpenAI()
tools = load_tools(["llm-math", "wikipedia"], llm=llm)
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               handle_parsing_errors=True, 
                               verbose=True)

In [ ]:
agent_executor.invoke({'input': "Qu'est-ce que 25% de 300?"})

### Ses propres outils

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplie deux entiers."""
    return first_int * second_int

print(multiply.name)
print(multiply.description)
print(multiply.args)

In [ ]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Ajoute deux entiers."
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    "Calcule la puissance d'un entier donné."
    return base**exponent

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4.1")

tools = [multiply, add, exponentiate]
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "Porter 3 à la puissance 5 et multiplier le résultat par la somme de douze et de trois, puis élever le tout au carré."
    }
)

In [ ]:
prompt.pretty_print()

### Assemblage d'agents

![Texte alternatif](images/promptulate.png "Promptulate")

## L'audio

### Text to speech

In [ ]:
from openai import OpenAI

response = OpenAI().audio.speech.create(
  model="tts-1-hd",
  voice="onyx", # alloy, onyx, fable, echo
  input="Coucou, comment allez-vous ?"
)
response.with_streaming_response.method('mon_audio.mp3')

### Speech to text

In [ ]:
from openai import OpenAI

OpenAI().audio.transcriptions.create(
  model="whisper-1", 
  file=open("mon_audio.mp3", "rb"),
  language="fr"
).text

In [ ]:
from openai import OpenAI
from pydub import AudioSegment

client = OpenAI()

song = AudioSegment.from_mp3("mon_fichier.mp3")
transcription = ''
for pas in range(0, 120, 20):
    debut, fin = pas * 60 * 1000, (pas+20) * 60 * 1000
    extrait = song[debut:fin]
    if len(extrait) > 100:
        extrait.export(f"extrait_{pas}.mp3", format="mp3")
        audio_file = open(f"extrait_{pas}.mp3", "rb")
        transcription += client.audio.transcriptions.create(
          model="whisper-1", 
          file=audio_file
        ).text

## Multimodal

In [33]:
import base64

def encode_image_to_data_url(image_path: str) -> str:
    """
    Lit l'image et renvoie une data URL prête à être insérée dans le payload.
    """
    # Lecture du fichier image
    with open(image_path, "rb") as img_file:
        image_bytes = img_file.read()  # Lecture binaire du contenu
    # Encodage base64
    b64 = base64.b64encode(image_bytes).decode("utf-8")
    # Détection du MIME type selon l'extension
    ext = os.path.splitext(image_path)[1].lower().lstrip(".")
    mime = f"image/{ext if ext != 'jpg' else 'jpeg'}"
    return f"data:{mime};base64,{b64}"

In [34]:
from openai import OpenAI

client = OpenAI()

data_url = encode_image_to_data_url("mon_image.png")
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Que vois-tu sur cette image ?"},
            {"type": "image_url", "image_url": {"url": data_url}}
        ]
    }
]

NameError: name 'os' is not defined

In [ ]:
response = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    max_tokens=500,
    temperature=0.0
)
response.choices[0].message.content